## **Time Series Prediction With Smoothing Methods**

The second part of Konrad's series deals with the application of vintage time series prediction methods, namely **Exponential Smoothing**, which have been around since the 1950s. According to Konrad's [NB](https://www.kaggle.com/code/konradb/ts-1a-smoothing-methods):
> ...(these) ideas were variations around the topic of exponential smoothing: weighted averages of past observations, with the weights decaying exponentially as the observations get older. A huge practical advantage of those methods that one only needs the previous smoothed value and current time series value to create a forecast for the next step - an important aspect in an era when a computer had less memory than a cheapest smartphone available in the market today.

> As computing power became more available, exponential smoothing methods started fading into the "historical" section of statistics guide - until people started realizing that with the latency requirements of trading or e-commerce, speed is more important that additional 5pct improvement in error and one-step forecasts started making a comeback.